In [1]:
import random
import numpy as np
import math
import time
import os

In [2]:
def euclidean_distance(a, b):
    return math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

In [11]:
def genetic_backpack(max_capacity, w, c):
    N = len(w)
    n_chromo = 100
    population = [[0 for j in range(N)] for i in range(n_chromo)] #np.zeros(N*n_chromo).reshape(n_chromo, N)
    k = 2
    # генерируем хромосомы
    for i in range(n_chromo):
        for j in range(N):
            population[i][j] = random.randint(0,1)
    #print('Популяция:\n', population)
    def remove_random_item(chromo):
        one_positions = []
        for i in range(len(chromo)):
            if chromo[i]:
                one_positions.append(i)
        chromo[one_positions[random.randint(0, len(one_positions) - 1)]] = 0
    
    def fitness_function(chromo, w, c):
            temp_w, temp_p = 0, 0
            for j in range(N):
                if chromo[j]:
                    temp_w += w[j]
                    temp_p += c[j]
            return temp_p, temp_w
    
    def crossover(chromos_selected):
            crossover_point = random.randint(1, N-1)
            tmp = chromos_selected[1][:crossover_point].copy()
            chromos_selected[1][:crossover_point], chromos_selected[0][:crossover_point] = chromos_selected[0][:crossover_point], tmp
            return chromos_selected
    
    def mutation(chromos_selected):
            for j in range(len(chromos_selected)):
                for i in range(len(chromos_selected[j])):
                    if random.randint(1, 50) == 1:
                        chromos_selected[j][i] = (chromos_selected[j][i] + 1) % 2
            return chromos_selected
    
    best_solution = [[], -1, -1]
    for i in range(100):
        # Считаем вес и стоимость
        total_chromo = []
        for i in range(n_chromo):
            temp_p, temp_w = fitness_function(population[i], w, c)

            while temp_w > max_capacity:
                remove_random_item(population[i])
                temp_p, temp_w = fitness_function(population[i], w, c)
            total_chromo.append((i, temp_p, temp_w))
        #print('Популяция после подсчета фит. функции:\n', population)  

        # Сортируем
        total_chromo = sorted(total_chromo, key=lambda x: x[1], reverse=True)
        #print(total_chromo[0])
        if best_solution[1] < total_chromo[0][1]:
            best_solution = [population[total_chromo[0][0]].copy(), total_chromo[0][1], total_chromo[0][2]]
            
        # Находим распределение вероятностей
        fitness_sum = 0
        for x in total_chromo:
            fitness_sum += x[1]
        distribution = []
        for chromo in population:
            distribution.append(fitness_function(chromo, w, c)[0] / fitness_sum)
        d = n_chromo // 10
        for i in range(n_chromo - d, n_chromo):
            distribution[i] *= 2
        best_chromos = random.choices(population, weights=distribution, k=k)
        
        new_population = []
        for i in range(n_chromo // 2 - 1):
            #print(population)
            chromos_selected = random.sample(population, 2)
            population.remove(chromos_selected[0])
            population.remove(chromos_selected[1])
            chromos_crossovered = crossover(chromos_selected)
            chromos_mutated = mutation(chromos_crossovered)
            new_population.append(chromos_mutated[0])
            new_population.append(chromos_mutated[1])

        new_population.append(best_chromos[0])
        new_population.append(best_chromos[1])
        
        population = new_population.copy()

    return best_solution

In [14]:
p_c = []
p_p = []
p_s = []
p_w = []
for i in range(1, 8):
    p_c.append(open(os.getcwd() + "\\benchmarks_knapsack\\p0" + str(i) + "_c.txt", "r"))
    p_p.append(open(os.getcwd() + "\\benchmarks_knapsack\\p0" + str(i) + "_p.txt", "r"))
    p_s.append(open(os.getcwd() + "\\benchmarks_knapsack\\p0" + str(i) + "_s.txt", "r"))
    p_w.append(open(os.getcwd() + "\\benchmarks_knapsack\\p0" + str(i) + "_w.txt", "r"))
for test_num in range(7):
    print(f"BENCHMARK p0{test_num+1}")
    w = int(p_c[test_num].read())
    items_c = []
    for line in p_p[test_num]:
        items_c.append(int(line))
        
    items_w = []
    for line in p_w[test_num]:
         items_w.append(int(line))

    items_true = []
    w_true = 0
    c_true = 0
    for line in p_s[test_num]:
        items_true.append(int(line))
    for j in range(len(items_true)):
        if items_true[j] == 1:         
            w_true += items_w[j]
            c_true += items_c[j]
                
    answers = []
    timeExecuted = 0
    n_iterations = 1
    for i in range(n_iterations):
        start = time.time()
        answers.append(genetic_backpack(w, items_w, items_c))
        timeExecuted += time.time() - start
    #answers.append(genetic_backpack(w, items_w, items_c))
    print(f"TIME: {timeExecuted/n_iterations * 1000}")
    bestResult = max(list(map(lambda x: x[1], answers)))
    print(f'COST: {bestResult}')
    for i in answers:
        if i[1] == bestResult:
            print(f'ITEMS: {i[0]}')
            print(f'WEIGHTS: {i[2]}\n\n')
            break

BENCHMARK p01
TIME: 126.9989013671875
COST: 309
ITEMS: [1, 1, 1, 1, 0, 1, 0, 0, 0, 0]
WEIGHTS: 165


BENCHMARK p02
TIME: 81.99763298034668
COST: 51
ITEMS: [0, 1, 1, 1, 0]
WEIGHTS: 26


BENCHMARK p03
TIME: 89.99896049499512
COST: 150
ITEMS: [1, 1, 0, 0, 1, 0]
WEIGHTS: 190


BENCHMARK p04
TIME: 98.9985466003418
COST: 107
ITEMS: [1, 0, 0, 1, 0, 0, 0]
WEIGHTS: 50


BENCHMARK p05
TIME: 111.00053787231445
COST: 900
ITEMS: [1, 0, 1, 1, 1, 0, 1, 1]
WEIGHTS: 104


BENCHMARK p06
TIME: 99.00069236755371
COST: 1735
ITEMS: [0, 1, 0, 1, 0, 0, 1]
WEIGHTS: 169


BENCHMARK p07
TIME: 192.99602508544922
COST: 1453
ITEMS: [1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1]
WEIGHTS: 749




In [15]:
def genetic_salesman(graph):
    n_edges = len(graph)
    n_chromo = 200 # должно быть четным
    k = 2
    population = []
    best_solution = [99999999, []]
    
    def fitness_function(chromo):
        res = 0
        for i in range(n_edges-1):
            res += graph[chromo[i]][chromo[i+1]]
        return res
    
    def crossover(chromos_selected):    
        sep = random.choices([i for i in range(n_edges)], k=2)
        while len(np.unique(sep, axis=0)) < k:
            sep = random.choices([i for i in range(n_edges)], k=2)
        sep.sort()
        chromos_crossovered = [[-1 for i in range(n_edges)], [-1 for i in range(n_edges)]]
        pairs = []
        collisions = [[],[]]
        for i in range(sep[0], sep[1]+1):
            pairs.append((chromos_selected[0][i], chromos_selected[1][i]))
            chromos_crossovered[0][i] = chromos_selected[1][i]
            chromos_crossovered[1][i] = chromos_selected[0][i]
            collisions[0].append(chromos_selected[1][i])
            collisions[1].append(chromos_selected[0][i])
        for i in range(n_edges):
            if chromos_selected[0][i] not in collisions[0] and chromos_crossovered[0][i] == -1:
                chromos_crossovered[0][i] = chromos_selected[0][i]
                collisions[0].append(chromos_selected[0][i])
            if chromos_selected[1][i] not in collisions[1] and chromos_crossovered[1][i] == -1:
                chromos_crossovered[1][i] = chromos_selected[1][i]
                collisions[1].append(chromos_selected[1][i])
        for i in range(n_edges):
            if chromos_selected[0][i] in collisions[0] and chromos_crossovered[0][i] == -1:
                temp = chromos_selected[0][i]
                temp_pairs = pairs.copy()
                while temp in collisions[0]:
                    for p in temp_pairs:
                        if temp == p[0]:
                            temp = p[1]
                            temp_pairs.remove(p)
                            break
                        elif temp == p[1]:
                            temp = p[0]
                            temp_pairs.remove(p)
                            break
                chromos_crossovered[0][i] = temp

            if chromos_selected[1][i] in collisions[1] and chromos_crossovered[1][i] == -1:
                temp = chromos_selected[1][i]
                temp_pairs = pairs.copy()
                while temp in collisions[1]:
                    for p in temp_pairs:
                        if temp == p[0]:
                            temp_pairs.remove(p)
                            temp = p[1]
                            break
                        elif temp == p[1]:
                            temp = p[0]
                            temp_pairs.remove(p)
                            break
                chromos_crossovered[1][i] = temp
        return chromos_crossovered
    
    def mutation(chromos_crossovered):
        for c in chromos_crossovered:
            if random.randint(0, 100) < 5:
                i, j = random.randint(0, n_edges-1), random.randint(0, n_edges-1)
                c[i], c[j] = c[j], c[i]
        return chromos_crossovered
    
    def initial_solution():
        for i in range(n_chromo):
            population.append([i for i in range(n_edges)])
            random.shuffle(population[i])
        return population
    
    for i in range(200):
        if i == 0:
            population = initial_solution()
        fitness_values = []
        fitness_sum = 0
        for i in range(n_chromo):
            ff = fitness_function(population[i])
            fitness_values.append((i, ff))
            fitness_sum += ff
        fitness_values.sort(key=lambda x: x[1])
        if fitness_values[0][1] <= best_solution[0]:
            best_solution = [fitness_values[0][1], population[fitness_values[0][0]].copy()]
        
        distribution = [0 for i in range(n_chromo)]
        for x in fitness_values:
            distribution[x[0]] = x[1] / fitness_sum

        best_chromos = random.choices(population, weights=distribution, k=k)
        while len(np.unique(best_chromos, axis=0)) < k:
            best_chromos = random.choices(population, weights=distribution, k=k)

        new_population = []
        for i in range(n_chromo // 2 - 1):
            chromos_selected = random.sample(population, 2)
            population.remove(chromos_selected[0])
            population.remove(chromos_selected[1])
            chromos_crossovered = crossover(chromos_selected)
            chromos_mutated = mutation(chromos_crossovered)
            new_population.append(chromos_mutated[0])
            new_population.append(chromos_mutated[1])

        new_population.append(best_chromos[0])
        new_population.append(best_chromos[1])

        population = new_population.copy()

    return best_solution

In [16]:
salesman_benchmarks = {'a280': open('benchmarks_salesman/a280.tsp', 'r').readlines()[6:-1],
                           'att48': open('benchmarks_salesman/att48.tsp', 'r').readlines()[6:-1],
                           'ch150': open('benchmarks_salesman/ch150.tsp', 'r').readlines()[6:-1],
                           'fl417': open('benchmarks_salesman/fl417.tsp', 'r').readlines()[6:-1],
                           'bays29': open('benchmarks_salesman/bays29.tsp', 'r').readlines()[38:-1],
                            'gr17': open('benchmarks_salesman/gr17.tsp', 'r').readlines()[7:-1],
                           }

for benchmark in salesman_benchmarks:
    if benchmark == 'gr17':
        total = []
        for line in salesman_benchmarks[benchmark]:
            for x in [float(x) for x in line.split()]:
                total.append(x)
        w = [[] for i in range(17)]
        count = 0
        for x in total:
            w[count].append(x)
            if x == 0:
                count += 1
        for line in w:
            for i in range(17 - len(line)):
                line.append(-1)
        for i in range(17):
            for j in range(17):
                if i != j:
                    w[i][j] = w[j][i]
    else:            
        coord = []
        for line in salesman_benchmarks[benchmark]:
            coord.append((float(line.split()[1]), float(line.split()[2])))
        w = []
        for i in range(len(coord)):
            w.append([])
            for j in range(len(coord)):
                w[i].append(euclidean_distance(coord[i], coord[j]))
                
    answers = []
    timeExecuted = 0
    n_iterations = 20
    for i in range(n_iterations):
        start = time.time()
        answer = genetic_salesman(w)
        timeExecuted += time.time() - start
        answers.append(answer)

    print(f"BENCHMARK {benchmark}")
    print(f"TIME {timeExecuted/n_iterations}")
    bestResult = min(list(map(lambda x: x[0], answers)))
    print(f'BEST RESULT: {bestResult}')
    for i in answers:
        if i[0] == bestResult:
            print(f'PATH: {i[1]}')
            break

BENCHMARK a280
TIME 69.97606158256531
BEST RESULT: 29355.003806563953
PATH: [231, 210, 244, 275, 112, 114, 111, 31, 213, 261, 259, 243, 248, 250, 130, 270, 175, 178, 68, 103, 80, 272, 1, 44, 167, 74, 185, 168, 177, 157, 97, 55, 58, 39, 89, 199, 180, 108, 242, 271, 197, 88, 82, 30, 3, 28, 184, 53, 179, 215, 194, 64, 107, 116, 245, 165, 102, 226, 236, 137, 195, 19, 254, 274, 118, 146, 176, 16, 42, 95, 92, 94, 75, 217, 173, 155, 0, 66, 38, 163, 277, 252, 37, 34, 132, 224, 133, 51, 120, 4, 279, 113, 87, 153, 86, 70, 238, 121, 54, 214, 208, 77, 49, 166, 191, 211, 256, 8, 205, 59, 151, 98, 258, 6, 10, 105, 159, 126, 14, 2, 131, 141, 7, 198, 263, 172, 12, 235, 228, 247, 189, 101, 109, 128, 135, 262, 104, 158, 201, 222, 160, 251, 138, 18, 134, 240, 169, 221, 9, 273, 78, 127, 241, 278, 99, 181, 33, 268, 196, 123, 17, 212, 61, 152, 183, 156, 60, 50, 56, 20, 79, 52, 147, 149, 136, 83, 76, 264, 22, 188, 209, 170, 110, 21, 72, 207, 67, 48, 71, 41, 225, 106, 187, 85, 190, 223, 202, 218, 227, 237, 26